**Manual Client-Server Simulation: ** \\

Instead of using tensorflow_federated, the code now simulates federated learning manually.  \\
Each client trains its model on decrypted data (simulating how the encrypted data would be processed if decryption happened during training).
After each round, the server aggregates the weights by averaging the client model weights.  \\

**Print Dataset:** \\

The plain dataset is printed first. The encrypted dataset is printed to demonstrate the Paillier encryption process. \\

**Training Process:** \\



Each client trains its model on its own (decrypted) data, and the server aggregates the weights after each round.  \\

The code prints the loss and accuracy for each client after training.
Metrics: \\

At the end of each round, we print out the loss and accuracy metrics for each client model. \\

In [21]:
import numpy as np
from phe import paillier

# Step 1: Paillier Encryption Key Generation
public_key, private_key = paillier.generate_paillier_keypair()

# Step 2: Dataset Creation (Encrypted)
def create_dataset():
    """
    Generate a dataset with random values for features and binary labels.
    Returns:
        x_data (ndarray): Feature data of shape (10, 2)
        y_data (ndarray): Binary labels of shape (10, 1)
    """
    x_data = np.random.rand(10, 2)  # 10 samples with 2 features
    y_data = np.random.randint(0, 2, size=(10, 1))  # Binary labels reshaped to (10, 1)
    return x_data, y_data

# Step 3: Encrypt Dataset
def encrypt_dataset(x_data, y_data, public_key):
    """
    Encrypts the dataset using the Paillier public key.
    Args:
        x_data (ndarray): Feature data
        y_data (ndarray): Label data
        public_key (PaillierPublicKey): Public key for encryption
    Returns:
        encrypted_x (list): Encrypted feature data
        encrypted_y (list): Encrypted label data
    """
    encrypted_x = [[public_key.encrypt(value) for value in row] for row in x_data]
    encrypted_y = [public_key.encrypt(int(label)) for label in y_data.flatten()]
    return encrypted_x, encrypted_y

# Step 4: Simulate Encrypted Operations
def encrypted_addition(enc_val1, enc_val2):
    """
    Perform encrypted addition using Paillier.
    Args:
        enc_val1 (EncryptedNumber): Encrypted value 1
        enc_val2 (EncryptedNumber): Encrypted value 2
    Returns:
        EncryptedNumber: Result of addition
    """
    return enc_val1 + enc_val2

def encrypted_multiply_constant(enc_val, constant):
    """
    Perform encrypted multiplication with a constant using Paillier.
    Args:
        enc_val (EncryptedNumber): Encrypted value
        constant (float): Constant to multiply
    Returns:
        EncryptedNumber: Result of multiplication
    """
    return enc_val * constant

# Step 5: Simulate a Dense Layer Forward Pass
def dense_layer_forward(enc_x_row, weights, bias, public_key):
    """
    Simulates a forward pass through a dense layer using encrypted inputs.
    Args:
        enc_x_row (list): Encrypted input features for one sample
        weights (list): Plaintext weights for the dense layer
        bias (float): Plaintext bias for the dense layer
        public_key (PaillierPublicKey): Public key for encryption
    Returns:
        EncryptedNumber: Encrypted output from the dense layer
    """
    # Simulate weighted sum (dot product) between encrypted inputs and weights
    weighted_sum = encrypted_addition(
        encrypted_multiply_constant(enc_x_row[0], weights[0]),
        encrypted_multiply_constant(enc_x_row[1], weights[1])
    )

    # Add bias (bias is plaintext, so we encrypt it)
    encrypted_bias = public_key.encrypt(bias)
    output = encrypted_addition(weighted_sum, encrypted_bias)

    # Apply a simple activation function approximation (like sigmoid or ReLU)
    # For this simulation, we'll just use multiplication by a constant as a simple approximation
    activated_output = encrypted_multiply_constant(output, 0.1)  # Simulate sigmoid approximation

    return activated_output

# Step 6: Train on Encrypted Data
def train_on_encrypted_data(encrypted_x, encrypted_y, public_key):
    """
    Simulates training on encrypted data. Since Paillier does not support
    certain operations like multiplying two encrypted numbers, this function
    uses conceptual approximations.
    Args:
        encrypted_x (list): Encrypted feature data
        encrypted_y (list): Encrypted label data
        public_key (PaillierPublicKey): Public key for encryption
    """
    # Define weights and bias for the dense layer (these are plaintext constants)
    weights = [0.5, 0.5]  # Two weights corresponding to two input features
    bias = 0.1  # Bias for the dense layer

    for epoch in range(10):  # Simulate 10 epochs
        print(f"\n--- Epoch {epoch + 1} ---")

        total_encrypted_loss = public_key.encrypt(0)  # Initialize encrypted loss as zero

        for enc_x_row, enc_y in zip(encrypted_x, encrypted_y):
            # Forward pass through the "dense layer"
            encrypted_output = dense_layer_forward(enc_x_row, weights, bias, public_key)

            # Calculate encrypted loss (mean squared error approximation)
            error = encrypted_output - enc_y
            encrypted_loss = encrypted_multiply_constant(error, error)  # Square the error (simulated)

            # Sum up the encrypted losses
            total_encrypted_loss = encrypted_addition(total_encrypted_loss, encrypted_loss)

        print(f"Total Encrypted Loss for Epoch {epoch + 1}: {total_encrypted_loss}")

# Step 7: Main Function to Execute the Process
def main():
    # Step 1: Create dataset
    x_data, y_data = create_dataset()

    # Step 2: Encrypt dataset
    encrypted_x, encrypted_y = encrypt_dataset(x_data, y_data, public_key)

    # Step 3: Train on encrypted data
    train_on_encrypted_data(encrypted_x, encrypted_y, public_key)

# Execute the main function
if __name__ == "__main__":
    main()



--- Epoch 1 ---


NotImplementedError: Good luck with that...

In [19]:
import numpy as np
import tensorflow as tf
from phe import paillier

# Step 1: Paillier Encryption Key Generation
# Generate a public and private key pair for encryption
public_key, private_key = paillier.generate_paillier_keypair()

# Step 2: Dataset Creation
# Function to create a dummy dataset with 2 features and binary labels
def create_dataset():
    x_data = np.random.rand(10, 2)  # 10 samples with 2 features
    y_data = np.random.randint(0, 2, size=(10,))  # Binary labels (0 or 1)
    return tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(2)  # Batch of size 2

# Step 3: Encryption Function
# Function to encrypt the dataset using Paillier encryption
def encrypt_dataset(dataset, public_key):
    encrypted_data = []
    for x, y in dataset:
        encrypted_x = [[public_key.encrypt(value) for value in row] for row in x.numpy()]  # Encrypt each feature
        encrypted_y = [public_key.encrypt(int(label)) for label in y.numpy()]  # Encrypt the labels
        encrypted_data.append((encrypted_x, encrypted_y))
    return encrypted_data

# Step 4: Decryption Function
# Decrypt the dataset back to its original form to simulate the training process
def decrypt_dataset(encrypted_data, private_key):
    decrypted_data = []
    for x, y in encrypted_data:
        decrypted_x = [[private_key.decrypt(value) for value in row] for row in x]  # Decrypt each feature
        decrypted_y = [private_key.decrypt(label) for label in y]  # Decrypt each label
        decrypted_data.append((decrypted_x, decrypted_y))
    return decrypted_data

# Step 5: Model Definition
# Define a simple neural network model for the federated learning process
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(2, activation='relu', input_shape=(2,)),  # Input layer with 2 features
        tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid for binary classification
    ])
    # Compile the model with optimizer, loss, and metrics
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 6: Federated Training Simulation for Scenario 2 (Decrypt Before Processing)
def federated_training_decrypt_train_encrypt():
    print("\nScenario 2: Decrypt Before Processing")
    clients_data = [create_dataset() for _ in range(3)]  # Create datasets for 3 clients
    encrypted_clients_data = [encrypt_dataset(client_data, public_key) for client_data in clients_data]  # Encrypt datasets

    models = [create_model() for _ in range(3)]  # Create separate models for each client

    for round_num in range(3):  # Simulate 3 rounds of training
        print(f"\n--- Round {round_num + 1} ---")
        client_weights = []

        for i, (encrypted_data, model) in enumerate(zip(encrypted_clients_data, models)):
            # Decrypt client data before training (this step is specific to Scenario 2)
            decrypted_data = decrypt_dataset(encrypted_data, private_key)

            # Train each client model on its decrypted data
            for x, y in decrypted_data:
                x_train = np.array(x, dtype=np.float32)
                y_train = np.array(y, dtype=np.float32).reshape(-1, 1)  # Reshape labels to match logits

                # Train the model using the decrypted data
                model.fit(x_train, y_train, epochs=1, verbose=0)

            # Store client model weights for averaging
            client_weights.append(model.get_weights())

            # Print client model metrics after training
            loss, acc = model.evaluate(x_train, y_train, verbose=0)
            print(f"Client {i+1} - Loss: {loss:.4f}, Accuracy: {acc:.4f}")

        # Server-side model aggregation: Average client weights
        new_weights = [np.mean([client_weights[i][layer] for i in range(3)], axis=0)
                       for layer in range(len(client_weights[0]))]

        # Re-encrypt the weights before sending them back to the clients (optional in Scenario 2)
        # In practice, this step would occur if the weights were sent back to the server for secure aggregation

        # Update all client models with the aggregated weights
        for model in models:
            model.set_weights(new_weights)

# Execute the Federated Training Process for Scenario 2
federated_training_decrypt_train_encrypt()



Scenario 2: Decrypt Before Processing

--- Round 1 ---
Client 1 - Loss: 0.7418, Accuracy: 0.5000
Client 2 - Loss: 0.6248, Accuracy: 1.0000
Client 3 - Loss: 0.5908, Accuracy: 1.0000

--- Round 2 ---
Client 1 - Loss: 0.6891, Accuracy: 0.5000
Client 2 - Loss: 0.7041, Accuracy: 0.0000
Client 3 - Loss: 0.7066, Accuracy: 0.0000

--- Round 3 ---
Client 1 - Loss: 0.6892, Accuracy: 0.5000
Client 2 - Loss: 0.7019, Accuracy: 0.0000
Client 3 - Loss: 0.7043, Accuracy: 0.0000


In [5]:
pip install tensorflow tensorflow-federated phe


Scenario 1: Federated Learning with Encrypted Input & Output (Fully Encrypted Training)

In [11]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from phe import paillier

# Step 1: Paillier Encryption Key Generation
public_key, private_key = paillier.generate_paillier_keypair()

# Step 2: Dataset Creation
def create_dataset():
    x_data = np.random.rand(10, 2)  # 10 samples with 2 features
    y_data = np.random.randint(0, 2, size=(10, 1))  # Binary labels reshaped to (10, 1)
    return tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(2)  # Batch size 2

# Step 3: Encryption Function
def encrypt_dataset(dataset, public_key):
    encrypted_data = []
    for x, y in dataset:
        encrypted_x = [[public_key.encrypt(value) for value in row] for row in x.numpy()]
        encrypted_y = [public_key.encrypt(int(label)) for label in y.numpy().flatten()]
        encrypted_data.append((encrypted_x, encrypted_y))
    return encrypted_data

# Step 4: Decryption Function
def decrypt_dataset(encrypted_data, private_key):
    decrypted_data = []
    for x, y in encrypted_data:
        decrypted_x = [[private_key.decrypt(value) for value in row] for row in x]
        decrypted_y = [private_key.decrypt(label) for label in y]
        decrypted_data.append((decrypted_x, decrypted_y))
    return decrypted_data

# Step 5: Define the Model Using Keras
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(2, activation='relu', input_shape=(2,)),  # Input layer with 2 features
        tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid for binary classification
    ])
    return model

# Step 6: Define a TFF model function for Federated Averaging
def model_fn():
    keras_model = create_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=(tf.TensorSpec(shape=[None, 2], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.float32)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Step 7: Federated Training Simulation for Scenario 1 (Fully Encrypted)
def federated_training_fully_encrypted():
    # Build Federated Averaging algorithm
    iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
        model_fn,
        client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
        server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
    )

    # Initialize the federated learning process
    state = iterative_process.initialize()

    clients_data = [create_dataset() for _ in range(3)]  # Create datasets for 3 clients

    # Encrypt client data
    encrypted_clients_data = [encrypt_dataset(client_data, public_key) for client_data in clients_data]

    # Simulate multiple training rounds
    for round_num in range(3):
        print(f"\n--- Round {round_num + 1} ---")

        # Decrypt the dataset for training
        decrypted_clients_data = [decrypt_dataset(encrypted_data, private_key) for encrypted_data in encrypted_clients_data]

        # Convert decrypted data back to TFF-compatible format
        def tff_data_format(decrypted_data):
            x = np.array([x for x, _ in decrypted_data], dtype=np.float32)
            y = np.array([y for _, y in decrypted_data], dtype=np.float32).reshape(-1, 1)  # Ensure (N, 1) shape for labels
            return tf.data.Dataset.from_tensor_slices((x, y)).batch(2)

        federated_data = [tff_data_format(decrypted_data) for decrypted_data in decrypted_clients_data]

        # Print the shapes of batches for debugging
        for client_idx, client_data in enumerate(federated_data):
            print(f"Client {client_idx+1}:")
            for batch in client_data:
                print(f"  Batch features shape: {batch[0].shape}, Batch labels shape: {batch[1].shape}")

        # Perform one round of federated training
        state, metrics = iterative_process.next(state, federated_data)
        print(f"Round {round_num + 1} - Metrics: {metrics}")

# Step 8: Execute the Federated Training Process
federated_training_fully_encrypted()



--- Round 1 ---


ValueError: Dimensions 5 and 10 are not compatible

Scenario 2: Federated Learning with Decryption Before Processing (Decrypted During Processing)

In [13]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from phe import paillier

# Step 1: Paillier Encryption Key Generation
# Generate a public and private key pair for encryption
public_key, private_key = paillier.generate_paillier_keypair()

# Step 2: Dataset Creation
# Function to create a dummy dataset with 2 features and binary labels
def create_dataset():
    x_data = np.random.rand(10, 2)  # 10 samples with 2 features
    y_data = np.random.randint(0, 2, size=(10,))  # Binary labels (0 or 1)
    return tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(2)  # Batch of size 2

# Step 3: Encryption Function
# Function to encrypt the dataset using Paillier encryption
def encrypt_dataset(dataset, public_key):
    encrypted_data = []
    for x, y in dataset:
        encrypted_x = [[public_key.encrypt(value) for value in row] for row in x.numpy()]  # Encrypt each feature
        encrypted_y = [public_key.encrypt(int(label)) for label in y.numpy()]  # Encrypt the labels
        encrypted_data.append((encrypted_x, encrypted_y))
    return encrypted_data

# Step 4: Decryption Function
# Decrypt the dataset back to its original form to simulate the training process
def decrypt_dataset(encrypted_data, private_key):
    decrypted_data = []
    for x, y in encrypted_data:
        decrypted_x = [[private_key.decrypt(value) for value in row] for row in x]  # Decrypt each feature
        decrypted_y = [private_key.decrypt(label) for label in y]  # Decrypt each label
        decrypted_data.append((decrypted_x, decrypted_y))
    return decrypted_data

# Step 5: Define Model Using Keras
# Create a simple sequential model for binary classification
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(2, activation='relu', input_shape=(2,)),  # Input layer with 2 features
        tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid for binary classification
    ])
    return model

# Step 6: Define a TFF model function for Federated Averaging
def model_fn():
    keras_model = create_model()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=(tf.TensorSpec(shape=[None, 2], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.float32)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Step 7: Federated Training Simulation for Scenario 2 (Decrypt Before Processing)
def federated_training_decrypt_before_processing():
    # Create the federated learning process
    iterative_process = tff.learning.build_federated_averaging_process(model_fn)
    state = iterative_process.initialize()

    clients_data = [create_dataset() for _ in range(3)]  # Create datasets for 3 clients

    # Encrypt client data
    encrypted_clients_data = [encrypt_dataset(client_data, public_key) for client_data in clients_data]

    # Simulate multiple training rounds
    for round_num in range(3):
        print(f"\n--- Round {round_num + 1} ---")

        # Decrypt the dataset before training
        decrypted_clients_data = [decrypt_dataset(encrypted_data, private_key) for encrypted_data in encrypted_clients_data]

        # Convert decrypted data back to TFF-compatible format
        def tff_data_format(decrypted_data):
            return tf.data.Dataset.from_tensor_slices(
                (np.array([x for x, y in decrypted_data], dtype=np.float32),
                 np.array([y for x, y in decrypted_data], dtype=np.float32).reshape(-1, 1))
            ).batch(2)

        federated_data = [tff_data_format(decrypted_data) for decrypted_data in decrypted_clients_data]

        # Perform one round of federated training
        state, metrics = iterative_process.next(state, federated_data)
        print(f"Round {round_num + 1} - Metrics: {metrics}")

# Step 8: Execute the Federated Training Process
federated_training_decrypt_before_processing()


AttributeError: module 'tensorflow_federated.python.learning' has no attribute 'build_federated_averaging_process'